# Yandex.Music

# Contents <a id='back'></a>

* [Introduction](#intro)
* [Stage 1. Data overview](#data_review)
    * [Conclusions](#data_review_conclusions)
* [Stage 2. Data preprocessing](#data_preprocessing)
    * [2.1 Header style](#header_style)
    * [2.2 Missing values](#missing_values)
    * [2.3 Duplicates](#duplicates)
    * [2.4 Conclusions](#data_preprocessing_conclusions)
* [Stage 3. Testing the hypotheses](#hypotheses)
    * [3.1 Hypothesis 1: user activity in the two cities](#activity)
    * [3.2 Hypothesis 2: music preferences on Monday and Friday](#week)
    * [3.3 Hypothesis 3: genre preferences in Springfield and Shelbyville](#genre)
* [Findings](#end)

## Introduction <a id='intro'></a>
Whenever we're doing research, we need to formulate hypotheses that we can then test. Sometimes we accept these hypotheses; other times, we reject them. To make the right decisions, a business must be able to understand whether or not it's making the right assumptions.

In this project, you'll compare the music preferences of the cities of Springfield and Shelbyville. You'll study real Yandex.Music data to test the hypotheses below and compare user behavior for these two cities.

### Goal: 
Test three hypotheses:
1. User activity differs depending on the day of the week and from city to city. 
2. On Monday mornings, Springfield and Shelbyville residents listen to different genres. This is also true for Friday evenings. 
3. Springfield and Shelbyville listeners have different preferences. In Springfield, they prefer pop, while Shelbyville has more rap fans.

### Stages 
Data on user behavior is stored in the file `/datasets/music_project_en.csv`. There is no information about the quality of the data, so you will need to explore it before testing the hypotheses. 

First, you'll evaluate the quality of the data and see whether its issues are significant. Then, during data preprocessing, you will try to account for the most critical problems.
 
Your project will consist of three stages:
 1. Data overview
 2. Data preprocessing
 3. Testing the hypotheses
 
[Back to Contents](#back)

## Stage 1. Data overview <a id='data_review'></a>

Open the data on Yandex.Music and explore it.

In [1]:
import pandas as pd


Read the file `music_project_en.csv` from the `/datasets/` folder and save it in the `df` variable:

In [2]:
df = pd.read_csv('/datasets/music_project_en.csv')
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63736,57512,63881,65079,65079,65079
unique,41748,39666,37806,268,2,20392,3
top,A8AE9169,Brand,Kartvelli,pop,Springfield,21:51:22,Friday
freq,76,136,136,8850,45360,14,23149


Print the first 10 table rows:

In [3]:
print(df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                       Chains          Obladaet  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE                     L’estate       Julia Dalia  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

        City        time        Day  
0  Shelbyville  20:28:33  Wednesday  
1  Springfield  14:07:09     Friday  
2  Shelbyville  20:58:07  Wednesday  
3  Shelbyville  08:37:09     Monday  
4  Springfield  08:34:34     Monday  
5

Obtaining the general information about the table with one command:

In [4]:
df.dtypes

  userID    object
Track       object
artist      object
genre       object
  City      object
time        object
Day         object
dtype: object

The table contains seven columns. They all store the same data type: `object`.

According to the documentation:
- `'userID'` — user identifier
- `'Track'` — track title
- `'artist'` — artist's name
- `'genre'`
- `'City'` — user's city
- `'time'` — the exact time the track was played
- `'Day'` — day of the week

We can see three issues with style in the column names:
1. Some names are uppercase, some are lowercase.
2. There are spaces in some names.
3. They are not all the same data type.

The number of column values is different. This means the data contains missing values.


### Conclusions <a id='data_review_conclusions'></a> 

Each row in the table stores data on a track that was played. Some columns describe the track itself: its title, artist and genre. The rest convey information about the user: the city they come from, the time they played the track. 

It's clear that the data is sufficient to test the hypotheses. However, there are missing values.

To move forward, we need to preprocess the data.

[Back to Contents](#back)

## Stage 2. Data preprocessing <a id='data_preprocessing'></a>
Correct the formatting in the column headers and deal with the missing values. Then, check whether there are duplicates in the data.

### Header style <a id='header_style'></a>
Print the column header:

In [5]:
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Change column names according to the rules of good style:
* If the name has several words, use snake_case
* All characters must be lowercase
* Delete spaces

In [6]:
df.rename(columns=({'  userID':'user_id','Track':'track', 'artist':'artist', 'genre':'genre', '  City  ':'city', 'time':'time', 'Day':'day'}), inplace=True,)

Check the result. Print the names of the columns once more:

In [7]:
print(df.head)

<bound method NDFrame.head of         user_id                              track            artist  \
0      FFB692EC                  Kamigata To Boots  The Mass Missile   
1      55204538        Delayed Because of Accident  Andreas Rönnberg   
2        20EC38                  Funiculì funiculà       Mario Lanza   
3      A3DD03C9              Dragons in the Sunset        Fire + Ice   
4      E2DC1FAE                        Soul People        Space Echo   
...         ...                                ...               ...   
65074  729CBB09                            My Name            McLean   
65075  D08D4A55  Maybe One Day (feat. Black Spade)       Blu & Exile   
65076  C5E3A0D5                          Jalopiina               NaN   
65077  321D0506                      Freight Train     Chas McDevitt   
65078  3A64EF84          Tell Me Sweet Little Lies      Monica Lopez   

            genre         city      time        day  
0            rock  Shelbyville  20:28:33  Wednesday

[Back to Contents](#back)

### Missing values <a id='missing_values'></a>
First, find the number of missing values in the table. To do so, use two `pandas` methods:

In [8]:
print(df.head(12))
print(df.isna().sum())

     user_id                                              track  \
0   FFB692EC                                  Kamigata To Boots   
1   55204538                        Delayed Because of Accident   
2     20EC38                                  Funiculì funiculà   
3   A3DD03C9                              Dragons in the Sunset   
4   E2DC1FAE                                        Soul People   
5   842029A1                                             Chains   
6   4CB90AA5                                               True   
7   F03E1C1F                                   Feeling This Way   
8   8FA1D3BE                                           L’estate   
9   E772D5C0                                          Pessimist   
10  BC5A3A29                                       Gool la Mita   
11  8B5192C0  Is There Anybody Out There? (Panoramic Paralys...   

                  artist       genre         city      time        day  
0       The Mass Missile        rock  Shelbyville  20:2

Not all missing values affect the research. For instance, the missing values in `track` and `artist` are not critical. You can simply replace them with clear markers.

But missing values in `'genre'` can affect the comparison of music preferences in Springfield and Shelbyville. In real life, it would be useful to learn the reasons why the data is missing and try to make up for them. But we do not have that opportunity in this project. So you will have to:
* Fill in these missing values with markers
* Evaluate how much the missing values may affect your computations

Replace the missing values in `'track'`, `'artist'`, and `'genre'` with the string `'unknown'`. To do this, create the `columns_to_replace` list, loop over it with `for`, and replace the missing values in each of the columns:

In [9]:
columns_to_replace = ['track', 'genre', 'artist']
for columns in columns_to_replace:
    df[columns] = df[columns].fillna('unknown')

Make sure the table contains no more missing values. Count the missing values again.

In [10]:
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


[Back to Contents](#back)

### Duplicates <a id='duplicates'></a>
Find the number of obvious duplicates in the table using one command:

In [11]:
print(df.duplicated().sum())

3826


Call the `pandas` method for getting rid of obvious duplicates:

In [12]:
df = df.drop_duplicates()
df = df.drop_duplicates().reset_index(drop=True)

Count obvious duplicates once more to make sure you have removed all of them:

In [13]:
print(df.duplicated().sum())

0


Now get rid of implicit duplicates in the `genre` column. For example, the name of a genre can be written in different ways. Such errors will also affect the result.

Print a list of unique genre names, sorted in alphabetical order. To do so:
* Retrieve the intended DataFrame column 
* Apply a sorting method to it
* For the sorted column, call the method that will return all unique column values

In [14]:
df.sort_values(by = ['genre'],inplace = True)
df['genre'].unique()
print(df['genre'])


31916         acid
3636      acoustic
20264     acoustic
37784     acoustic
14611     acoustic
           ...    
6158         world
17963        world
27939    worldbeat
39063    worldbeat
8448           ïîï
Name: genre, Length: 61253, dtype: object


Look through the list to find implicit duplicates of the genre `hiphop`. These could be names written incorrectly or alternative names of the same genre.

You will see the following implicit duplicates:
* `hip`
* `hop`
* `hip-hop`

To get rid of them, declare the function `replace_wrong_genres()` with two parameters: 
* `wrong_genres=` — the list of duplicates
* `correct_genre=` — the string with the correct value

The function should correct the names in the `'genre'` column from the `df` table, i.e. replace each value from the `wrong_genres` list with the value in `correct_genre`.

In [15]:
def replace_wrong_genres(wrong_genres, correct_genre):
    for genre in df:
        genre.replace(wrong_genres, correct_genre)

Call `replace_wrong_genres()` and pass it arguments so that it clears implicit duplcates (`hip`, `hop`, and `hip-hop`) and replaces them with `hiphop`:

In [16]:
replace_wrong_genres('hip', 'hiphop')
replace_wrong_genres('hop', 'hiphop')
replace_wrong_genres('hip-hop', 'hiphop')

Make sure the duplicate names were removed. Print the list of unique values from the `'genre'` column:

In [17]:
print(df['genre'].unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 'fitness' 'flamenco' 'folk'
 'folklore' 'folkmetal' 'folkrock' 'folktronica' 'forró' 'frankreich'
 'französisch' 

[Back to Contents](#back)

### Conclusions <a id='data_preprocessing_conclusions'></a>
We detected three issues with the data:

- Incorrect header styles
- Missing values
- Obvious and implicit duplicates

The headers have been cleaned up to make processing the table simpler.

All missing values have been replaced with `'unknown'`. But we still have to see whether the missing values in `'genre'` will affect our calculations.

The absence of duplicates will make the results more precise and easier to understand.

Now we can move on to testing hypotheses. 

[Back to Contents](#back)

## Stage 3. Testing hypotheses <a id='hypotheses'></a>

### Hypothesis 1: comparing user behavior in two cities <a id='activity'></a>

According to the first hypothesis, users from Springfield and Shelbyville listen to music differently. Test this using the data on three days of the week: Monday, Wednesday, and Friday.

* Divide the users into groups by city.
* Compare how many tracks each group played on Monday, Wednesday, and Friday.


For the sake of practice, perform each computation separately. 

Evaluate user activity in each city. Group the data by city and find the number of songs played in each group.



In [18]:
dfspringfield = df.query("city == 'Springfield'")
dfshelbyville = df.query("city == 'Shelbyville'")

Springfield has more tracks played than Shelbyville. But that does not imply that citizens of Springfield listen to music more often. This city is simply bigger, and there are more users.

Now group the data by day of the week and find the number of tracks played on Monday, Wednesday, and Friday.


In [19]:
dfspringfield_mon = dfspringfield.query("day == 'Monday'")
dfspringfield_wed = dfspringfield.query("day == 'Wednesday'")
dfspringfield_fri = dfspringfield.query("day == 'Friday'")
dfshelbyville_mon = dfshelbyville.query("day == 'Monday'")
dfshelbyville_wed = dfshelbyville.query("day == 'Wednesday'")
dfshelbyville_fri = dfshelbyville.query("day == 'Friday'")

Wednesday is the quietest day overall. But if we consider the two cities separately, we might come to a different conclusion.

You have seen how grouping by city or day works. Now write a function that will group by both.

Create the `number_tracks()` function to calculate the number of songs played for a given day and city. It will require two parameters:
* day of the week
* name of the city

In the function, use a variable to store the rows from the original table, where:
  * `'day'` column value is equal to the `day` parameter
  * `'city'` column value is equal to the `city` parameter

Apply consecutive filtering with logical indexing.

Then calculate the `'user_id'` column values in the resulting table. Store the result to a new variable. Return this variable from the function.

In [20]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

In [21]:
dataframe = df
day_of_the_wk_str = 'Friday'
city = 'Springfield'

#filter the input dataframe with input filters
df_temp = dataframe[(dataframe['day']==day_of_the_wk_str) & (dataframe['city']==city) ]
#for gross count
song_count = len(df_temp)

In [22]:
def get_song_count (dataframe, day_of_the_wk_str, city):
    #filter the input dataframe with input filters
    df_temp = dataframe[(dataframe['day']==day_of_the_wk_str) & (dataframe['city']==city) ]
    #for gross count
    song_count = len(df_temp)
    #for unique count
    song_count_unique = len(df_temp[['track','artist','time']].drop_duplicates)
    return 
    
def number_tracks(df, day_of_the_wk_str, city):
    df_temp = dataframe[(dataframe['day']==day_of_the_wk_str) & (dataframe['city']==city) ]
    song_count = len(df_temp)
    return song_count

Call `number_tracks()` six times, changing the parameter values, so that you retrieve the data on both cities for each of the three days.

In [23]:
# the number of songs played in Springfield on Monday
number_tracks (df, 'Monday', 'Springfield')

15740

In [24]:
# the number of songs played in Shelbyville on Monday
number_tracks (df, 'Monday', 'Shelbyville')

5614

In [25]:
# the number of songs played in Springfield on Wednesday
number_tracks (df, 'Wednesday', 'Springfield')

11056

In [26]:
# the number of songs played in Shelbyville on Wednesday
number_tracks (df, 'Wednesday', 'Shelbyville')

7003

In [27]:
# the number of songs played in Springfield on Friday
number_tracks (df, 'Friday', 'Springfield')

15945

In [28]:
# the number of songs played in Shelbyville on Friday
number_tracks (df, 'Friday', 'Shelbyville')

5895

Use `pd.DataFrame` to create a table, where
* Column names are: `['city', 'monday', 'wednesday', 'friday']`
* The data is the results you got from `number_tracks()`

In [29]:
# table with results
pd.DataFrame(data= [['Springfield',15740,11056,15945],['Shelbyville',5614,7003,5895]], columns= ['city', 'monday', 'wednesday', 'friday'])

,city,monday,wednesday,friday
0,Springfield,15740,11056,15945
1,Shelbyville,5614,7003,5895


**Conclusions**

The data reveals differences in user behavior:

- In Springfield, the number of songs played peaks on Mondays and Fridays, while on Wednesday there is a decrease in activity.
- In Shelbyville, on the contrary, users listen to music more on Wednesday. User activity on Monday and Friday is smaller.

So the first hypothesis seems to be correct.

[Back to Contents](#back)

### Hypothesis 2: music at the beginning and end of the week <a id='week'></a>

According to the second hypothesis, on Monday morning and Friday night, citizens of Springfield listen to genres that differ from ones users from Shelbyville enjoy.

Get tables (make sure that the name of your combined table matches the DataFrame given in the two code blocks below):
* For Springfield — `spr_general`
* For Shelbyville — `shel_general`

In [30]:
# create the spr_general table from the df rows, 
# where the value in the 'city' column is 'Springfield'
spr_general = df.query("city == 'Springfield'")


In [31]:
# create the shel_general from the df rows,
print(df.head(5))
# where the value in the 'city' column is 'Shelbyville'
shel_general = df.query("city == 'Shelbyville'")

        user_id                       track            artist     genre  \
31916  C40BC5DF            Shot in the Dark        Four80East      acid   
3636   F10919ED          Ready For The Fire  Valley Of Wolves  acoustic   
20264  B02ED3AB  Prelude – Ne Me Qui Te Pas           unknown  acoustic   
37784  77D3A7D9  Prelude – Ne Me Qui Te Pas           unknown  acoustic   
14611  1C7D9E90  Prelude – Ne Me Qui Te Pas           unknown  acoustic   

              city      time        day  
31916  Springfield  09:44:12     Friday  
3636   Shelbyville  21:33:18     Monday  
20264  Springfield  13:31:03  Wednesday  
37784  Springfield  14:27:34     Monday  
14611  Shelbyville  21:47:42     Friday  


Write the `genre_weekday()` function with four parameters:
* A table for data (`df`)
* The day of the week (`day`)
* The first timestamp, in 'hh:mm' format (`time1`)
* The last timestamp, in 'hh:mm' format (`time2`)

The function should return info on the 15 most popular genres on a given day within the period between the two timestamps.

In [32]:
# 1) Let the genre_df variable store the rows that meet several conditions:
df['time'] = pd.to_datetime(df['time'], infer_datetime_format=True)
df = dataframe
#    - the value in the 'day' column is equal to the value of the day= argument
day = 'friday'
#    - the value in the 'time' column is greater than the value of the time1= argument
time1 = df['time'].dt.hour
#    - the value in the 'time' column is smaller than the value of the time2= argument
time2 = df['time'].dt.hour
#    Use consecutive filtering with logical indexing.
    #genre_df = df[(df['day']==day) & (df['hh:mm']>=time1) & (df['hh:mm']<=time2)]
# 2) Group genre_df by the 'genre' column, take one of its columns, 
#    and use the count() method to find the number of entries for each of 
#    the represented genres; store the resulting Series to the
#    genre_df_count variable
    #genre_df_count = genre_df.groupby('genre').count()
# 3) Sort genre_df_count in descending order of frequency and store the result
#    to the genre_df_sorted variable
    #genre_df_sorted = genre_df_count.sort_values(ascending=False)
# 4) Return a Series object with the first 15 genre_df_sorted value - the 15 most
#    popular genres (on a given day, within a certain timeframe)
    #genre_df_sorted.head(15)
# Write your function here
def genre_weekday(df, day, time1, time2):
    # consecutive filtering
    # Create the variable genre_df which will store only those df rows where the day is equal to day=
    genre_df = df.loc[(df['day']==day)]

    # filter again so that genre_df will store only those rows where the time is smaller than time2=
    genre_df = genre_df.loc[(genre_df['time']<time2)]

    # filter once more so that genre_df will store only rows where the time is greater than time1=
    genre_df = genre_df.loc[(genre_df['time']>time1)]

    # group the filtered DataFrame by the column with the names of genres, take the genre column, and find the number of rows for each genre with the count() method
    genre_df_count = genre_df.groupby('genre').count()

    # sort the result in descending order (so that the most popular genres come first in the Series object)
    genre_df_sorted = genre_df_count['user_id'].sort_values(ascending=False)

    # we will return the Series object storing the 15 most popular genres on a given day in a given timeframe
    return genre_df_sorted[:15]

Compare the results of the `genre_weekday()` function for Springfield and Shelbyville on Monday morning (from 7AM to 11AM) and on Friday evening (from 17:00 to 23:00):

In [33]:
# calling the function for Monday morning in Springfield (use spr_general instead of the df table)
genre_weekday(spr_general, 'Monday', '07:00:00', '11:00:00')

genre
pop            781
dance          549
electronic     480
rock           474
hip            281
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: user_id, dtype: int64

In [35]:
# calling the function for Monday morning in Shelbyville (use shel_general instead of the df table)
genre_weekday(shel_general, 'Monday', '07:00:00', '11:00:00')

genre
pop            218
dance          182
rock           162
electronic     147
hip             79
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: user_id, dtype: int64

In [36]:
# calling the function for Friday evening in Springfield
genre_weekday(spr_general, 'Friday', '17:00:00', '23:00:00')

genre
pop            713
rock           517
dance          495
electronic     482
hip            267
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: user_id, dtype: int64

In [37]:
# calling the function for Friday evening in Shelbyville
genre_weekday(shel_general, 'Friday', '17:00:00', '23:00:00')

genre
pop            256
electronic     216
rock           216
dance          210
hip             94
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: user_id, dtype: int64

**Conclusion**

Having compared the top 15 genres on Monday morning, we can draw the following conclusions:

1. Users from Springfield and Shelbyville listen to similar music. The top five genres are the same, only rock and electronic have switched places.

2. In Springfield, the number of missing values turned out to be so big that the value `'unknown'` came in 10th. This means that missing values make up a considerable portion of the data, which may be a basis for questioning the reliability of our conclusions.

For Friday evening, the situation is similar. Individual genres vary somewhat, but on the whole, the top 15 is similar for the two cities.

Thus, the second hypothesis has been partially proven true:
* Users listen to similar music at the beginning and end of the week.
* There is no major difference between Springfield and Shelbyville. In both cities, pop is the most popular genre.

However, the number of missing values makes this result questionable. In Springfield, there are so many that they affect our top 15. Were we not missing these values, things might look different.

[Back to Contents](#back)

### Hypothesis 3: genre preferences in Springfield and Shelbyville <a id='genre'></a>

Hypothesis: Shelbyville loves rap music. Springfield's citizens are more into pop.

Group the `spr_general` table by genre and find the number of songs played for each genre with the `count()` method. Then sort the result in descending order and store it to `spr_genres`.

In [38]:
# on one line: group the spr_general table by the 'genre' column, 
# count the 'genre' values with count() in the grouping, 
spr_genres = spr_general.groupby('genre')['user_id'].count().sort_values(ascending= False)
# sort the resulting Series in descending order, and store it to spr_genres

Print the first 10 rows from `spr_genres`:

In [39]:
# printing the first 10 rows of spr_genres
print(spr_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hip            2041
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: user_id, dtype: int64


Now do the same with the data on Shelbyville.

Group the `shel_general` table by genre and find the number of songs played for each genre. Then sort the result in descending order and store it to the `shel_genres` table:


In [40]:
# on one line: group the shel_general table by the 'genre' column, 
# count the 'genre' values in the grouping with count(), 
# sort the resulting Series in descending order and store it to shel_genres
shel_genres = shel_general.groupby('genre')['city'].count().sort_values(ascending= False)

Print the first 10 rows of `shel_genres`:

In [41]:
# printing the first 10 rows from shel_genres
print(shel_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hip             934
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: city, dtype: int64


**Conclusion**

The hypothesis has been partially proven true:
* Pop music is the most popular genre in Springfield, as expected.
* However, pop music turned out to be equally popular in Springfield and Shelbyville, and rap wasn't in the top 5 for either city.


[Back to Contents](#back)

# Findings <a id='end'></a>

We have tested the following three hypotheses:

1. User activity differs depending on the day of the week and from city to city. 
2. On Monday mornings, Springfield and Shelbyville residents listen to different genres. This is also true for Friday evenings. 
3. Springfield and Shelbyville listeners have different preferences. In both Springfield and Shelbyville, they prefer pop.

After analyzing the data, we concluded:

1. User activity in Springfield and Shelbyville depends on the day of the week, though the cities vary in different ways. 

The first hypothesis is fully accepted.

2. Musical preferences do not vary significantly over the course of the week in both Springfield and Shelbyville. We can see small differences in order on Mondays, but:
* In Springfield and Shelbyville, people listen to pop music most.

So we can't accept this hypothesis. We must also keep in mind that the result could have been different if not for the missing values.

3. It turns out that the musical preferences of users from Springfield and Shelbyville are quite similar.

The third hypothesis is rejected. If there is any difference in preferences, it cannot be seen from this data.

### Note 
In real projects, research involves statistical hypothesis testing, which is more precise and more quantitative. Also note that you cannot always draw conclusions about an entire city based on the data from just one source.

You will study hypothesis testing in the sprint on statistical data analysis.

[Back to Contents](#back)